# 使用掃掠作業微調超參數

有許多機器學習演算法需要超參數 (會影響定型的參數值，但無法從定型資料本身) 決定。 例如，在定型羅吉斯回歸模型時，您可以使用正規化速率超參數來計數器模型中的偏差;或定型卷積神經網路時，您可以使用學習速率和批次大小等超參數來控制權數的調整方式，以及分別在迷你批次中處理多少資料項目。 選擇超參數值可能會大幅影響定型模型的效能，或定型模型所花費的時間;而且通常需要嘗試多個組合來尋找最佳解決方案。 

## 開始之前

您將需要最新版的  **azureml-ai-ml** 套件，才能在此筆記本中執行程式碼。 執行下列資料格以確認已安裝它。

> **注意**：
> 如果未安裝 **azure-ai-ml** 套件，請執行 `pip install azure-ai-ml` 以安裝它。

In [ ]:
## 連線到您的工作區

安裝必要的 SDK 套件之後，現在您已準備好連線到您的工作區。

若要連線到工作區，我們需要識別碼參數 - 訂用帳戶識別碼、資源組名和工作區名稱。 資源組名和工作區名稱已為您填入。 您只需要訂用帳戶識別碼才能完成命令。

若要尋找必要的參數，請按一下 Studio 右上方的訂用帳戶和工作區名稱。 窗格會在右側開啟。

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 複製訂用帳戶識別碼，並將 **YOUR-SUBSCRIPTION-ID** 取代為您複製的值。 </p>

## 建立訓練指令碼
當您想要定型機器學習模型，但會改變輸入參數時，超參數微調很理想。 您必須建立預期輸入參數代表其中一個演算法超參數的定型腳本。

執行下列儲存格來建立 **src** 資料夾和定型腳本。

請注意，定型腳本需要兩個輸入參數：

- `--training_data` 其需要字串。 您將指定已註冊資料資產的路徑作為輸入定型資料。
- `--reg_rate` 其需要數位，但預設值為 `0.01` 。 您將使用此輸入參數進行超參數微調。

In [ ]:
## 設定和執行命令作業

執行下列資料格來定型分類模型以預測糖尿病。 模型是藉由執行**可在 src**資料夾中找到的**定型 py 腳本來定 \. **型。 它會使用已註冊 `diabetes-data` 的資料資產作為定型資料。 

- `code`：指定要執行之腳本的資料夾。
- `command`：指定要完全執行的內容。
- `environment`：指定要在計算上安裝的必要套件，再執行 命令。
- `compute`：指定要用來執行命令的計算。
- `display_name`：個別作業的名稱。
- `experiment_name`：作業所屬的實驗名稱。

請注意，命令作業只會執行定型腳本一次，且正規化速率為 `0.1` 。 在執行掃掠作業以微調超參數之前，最好先測試腳本是否如預期般運作命令作業。

In [ ]:
## 定義搜尋空間

當命令作業成功完成時，您可以設定和執行掃掠作業。 

首先，您必須指定超參數的搜尋空間。 若要定型三個模型，每個模型都有不同的正規化速率 (`0.01` 、 `0.1` 或 `1`) ，您可以使用超參數來定義搜尋空間 `Choice` 。 

## 設定並提交掃掠作業

您將使用掃掠函式對定型腳本執行超參數微調。 若要設定掃掠作業，您必須設定下列專案：

- `compute`：要執行作業的計算目標名稱。
- `sampling_algorithm`：要用於搜尋空間的超參數取樣演算法。 允許的值為：`random`、`grid` 和 `bayesian`。
- `primary_metric`：每個試用作業所報告的主要計量名稱。 您必須使用 `mlflow.log_metric()` 與相同的對應計量名稱，在使用者的定型指令碼中記錄計量。
- `goal`：的 `primary_metric` 優化目標。 允許的值有 `maximize` 與 `minimize`。
- `limits`：掃掠作業的限制。 例如，您想要定型的試用版或模型數量上限。

請注意，命令作業會作為掃掠作業的基底。 命令作業的組態將會由掃掠作業重複使用。

In [ ]:
執行下列儲存格以提交掃掠作業。

In [ ]:
作業完成時，流覽至作業概觀。 [ **試用版** ] 索引標籤會顯示所有已定型的模型，以及 `Accuracy` 您嘗試的每個正規化速率值分數有何差異。

## Configure and run a command job

Run the cell below to train a classification model to predict diabetes. The model is trained by running the **train\.py** script that can be found in the **src** folder. It uses the registered `diabetes-data` data asset as the training data. 

- `code`: specifies the folder that includes the script to run.
- `command`: specifies what to run exactly.
- `environment`: specifies the necessary packages to be installed on the compute before running the command.
- `compute`: specifies the compute to use to run the command.
- `display_name`: the name of the individual job.
- `experiment_name`: the name of the experiment the job belongs to.

Note that the command job only runs the training script once, with a regularization rate of `0.1`. Before you run a sweep job to tune hyperparameters, it's a best practice to test whether your script works as expected with a command job.

In [ ]:
from azure.ai.ml import command, Input
from azure.ai.ml.constants import AssetTypes

# configure job

job = command(
    code="./src",
    command="python train.py --training_data ${{inputs.diabetes_data}} --reg_rate ${{inputs.reg_rate}}",
    inputs={
        "diabetes_data": Input(
            type=AssetTypes.URI_FILE, 
            path="azureml:diabetes-data:1"
            ),
        "reg_rate": 0.01,
    },
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-train-mlflow",
    experiment_name="diabetes-training", 
    tags={"model_type": "LogisticRegression"}
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

## Define the search space

When your command job has completed successfully, you can configure and run a sweep job. 

First, you'll need to specify the search space for your hyperparameter. To train three models, each with a different regularization rate (`0.01`, `0.1`, or `1`), you can define the search space with a `Choice` hyperparameter. 

In [ ]:
from azure.ai.ml.sweep import Choice

command_job_for_sweep = job(
    reg_rate=Choice(values=[0.01, 0.1, 1]),
)

## Configure and submit the sweep job

You'll use the sweep function to do hyperparameter tuning on your training script. To configure a sweep job, you'll need to configure the following:

- `compute`: Name of the compute target to execute the job on.
- `sampling_algorithm`: The hyperparameter sampling algorithm to use over the search space. Allowed values are `random`, `grid` and `bayesian`.
- `primary_metric`: The name of the primary metric reported by each trial job. The metric must be logged in the user's training script using `mlflow.log_metric()` with the same corresponding metric name.
- `goal`: The optimization goal of the `primary_metric`. The allowed values are `maximize` and `minimize`.
- `limits`: Limits for the sweep job. For example, the maximum amount of trials or models you want to train.

Note that the command job is used as the base for the sweep job. The configuration for the command job will be reused by the sweep job.

In [ ]:
# apply the sweep parameter to obtain the sweep_job
sweep_job = command_job_for_sweep.sweep(
    compute="aml-cluster",
    sampling_algorithm="grid",
    primary_metric="training_accuracy_score",
    goal="Maximize",
)

# set the name of the sweep job experiment
sweep_job.experiment_name="sweep-diabetes"

# define the limits for this sweep
sweep_job.set_limits(max_total_trials=4, max_concurrent_trials=2, timeout=7200)

Run the following cell to submit the sweep job.

In [ ]:
returned_sweep_job = ml_client.create_or_update(sweep_job)
aml_url = returned_sweep_job.studio_url
print("Monitor your job at", aml_url)

When the job is completed, navigate to the job overview. The **Trials** tab will show all models that have been trained and how the `Accuracy` score differs for each regularization rate value you tried.